In [1]:
import tensorflow as tf

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def pipe(data, batch_size = 128, shuffle = False):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    if shuffle:
        dataset = dataset.shuffle(buffer_size = batch_size * 10)
    dataset = dataset.batch(batch_size)
    #dataset = dataset.prefetch((batch_size * 2) + 1)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

(tr_x, tr_y), (te_x, te_y) = tf.keras.datasets.cifar10.load_data()

tr_x = tr_x * 1/255
te_x = te_x * 1/255

batch_size = 128

tr_data = pipe((tr_x, tr_y), batch_size = batch_size, shuffle = True)
te_data = pipe((te_x, te_y), batch_size = batch_size, shuffle = False)

170508288/170498071 [==============================] - 5s 0us/step


In [2]:
def get_model(shape = (32, 32, 3), weights = "imagenet"):
    model = tf.keras.applications.ResNet50(input_shape = shape, include_top = False, weights = weights)
    flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
    drop_out = tf.keras.layers.Dropout(0.5)(flatten)
    dense = tf.keras.layers.Dense(2048, activation = "relu")(drop_out)
    prediction = tf.keras.layers.Dense(10, activation = "softmax", name = "prediction")(dense)
    model = tf.keras.Model(model.input, prediction)
    return model

result = {}
for index in range(5):
    print("{0} - model learn".format(index + 1))
    model = get_model()
    loss = tf.keras.losses.sparse_categorical_crossentropy
    opt = tf.keras.optimizers.Adam(1e-4)
    metric = [tf.keras.metrics.sparse_categorical_accuracy]
    model.compile(loss = loss, optimizer = opt, metrics = metric)
    model.fit(tr_data, epochs = 10, verbose = 0)

    loss, score = model.evaluate(te_data, verbose = 1)
    path = "model{0}.h5".format(index + 1)

    result[path] = score
    model.save_weights(path)
print("finish")

1 - model learn
79/79 [==============================] - 3s 25ms/step - loss: 0.9806 - sparse_categorical_accuracy: 0.7928
2 - model learn
79/79 [==============================] - 2s 19ms/step - loss: 0.8892 - sparse_categorical_accuracy: 0.7996
3 - model learn
79/79 [==============================] - 3s 19ms/step - loss: 0.9653 - sparse_categorical_accuracy: 0.7863
4 - model learn
79/79 [==============================] - 2s 19ms/step - loss: 0.9529 - sparse_categorical_accuracy: 0.7896
5 - model learn
79/79 [==============================] - 2s 19ms/step - loss: 0.9828 - sparse_categorical_accuracy: 0.7956
finish


In [3]:
import os
import numpy as np
import model_soup

print("[Original Performance]")
for k, v in result.items():
    print("[{0}] accuracy:{1:.4f}".format(os.path.basename(k), v))

print("\n[Greedy Soup (uniform weight update) Performance]") #original paper style
greedy_model = model_soup.tf.greedy_soup(model, list(result.keys()), te_data, metric = tf.keras.metrics.sparse_categorical_accuracy, compare = np.greater_equal, update_greedy = False)
loss, score = greedy_model.evaluate(te_data)

print("\n[Greedy Soup (greedy weight update) Performance]")
greedy_model = model_soup.tf.greedy_soup(model, list(result.keys()), te_data, metric = tf.keras.metrics.sparse_categorical_accuracy, compare = np.greater_equal, update_greedy = True)
loss, score = greedy_model.evaluate(te_data)

print("\n[Uniform Soup Performance]")
uniform_model = model_soup.tf.uniform_soup(model, list(result.keys()))
loss, score = uniform_model.evaluate(te_data)

[Original Performance]
[model1.h5] accuracy:0.7928
[model2.h5] accuracy:0.7996
[model3.h5] accuracy:0.7863
[model4.h5] accuracy:0.7896
[model5.h5] accuracy:0.7956

[Greedy Soup (uniform weight update) Performance]
[model1.h5] step: 79 - time: 7.37s - sparse_categorical_accuracy: 0.7928
[model2.h5] step: 79 - time: 7.77s - sparse_categorical_accuracy: 0.8137
[model3.h5] step: 79 - time: 7.43s - sparse_categorical_accuracy: 0.8167
[model4.h5] step: 79 - time: 7.85s - sparse_categorical_accuracy: 0.8154
[model5.h5] step: 79 - time: 7.47s - sparse_categorical_accuracy: 0.8139
model soup best score : 0.8167
79/79 [==============================] - 2s 20ms/step - loss: 0.8823 - sparse_categorical_accuracy: 0.8167

[Greedy Soup (greedy weight update) Performance]
[model1.h5] step: 79 - time: 7.34s - sparse_categorical_accuracy: 0.7928
[model2.h5] step: 79 - time: 7.37s - sparse_categorical_accuracy: 0.8137
[model3.h5] step: 79 - time: 7.72s - sparse_categorical_accuracy: 0.8177
[model4.h5] st